<h2>Centerline</h2>

In this project, we use a centerline during the training phase to help the agent to avoid walls and to follow the track.

One way of defining such a trajectory is to make a human play the track safetly and save car's data at regular timesteps.

We also use states of the game to rewind the car at a random positions on the track.

In [1]:
import os
import time
import pickle
import keyboard
import numpy as np
import random

import tminterface as tmi
from tminterface.interface import TMInterface, Client

In [2]:
# Instantiate the Interface
# The client is used to connect the interface to the TMI server
# The interface runs itself in a separate thread

interface = TMInterface()
client = Client()

interface.register(client)

True

In [3]:
state = interface.get_simulation_state()

In [4]:
# HELPERS 

# Dynamic state of the car. Position, rotation, speed, etc...
#print(state.dyna.current_state)

# Data about wheels (index 1 out of 4). Contact with the ground, sliding...
#print(state.simulation_wheels[0])

# Time in milisseconds
#print(state.time / 1000 / 60)

In [5]:
TRACK_DATA_PATH = "track_data/"
track_name = "Straight_Line"

def create_folder_path(track_name):
    
    track_folder = os.path.join(TRACK_DATA_PATH, track_name)
    
    if os.path.exists(track_folder):
        n_run = len(os.listdir(track_folder))
    else:
        os.makedirs(track_folder)
        n_run = 0
    
    run_folder = os.path.join(track_folder, 'run-%s' % (n_run + 1))
    
    os.makedirs(run_folder)
    
    return run_folder

## Main Loop used to capture game data

In [6]:
run_folder = create_folder_path(track_name=track_name)

# Time in milisseconds
save_position_every = 10
save_state_every = 5_000

state = interface.get_simulation_state()
# Wait for the user to move to start saving data
print("Waiting for player to move")
while state.race_time >= 0:
    state = interface.get_simulation_state()

interface.execute_command("warp 0")
state = interface.get_simulation_state()

# Setup initial position and state just after the player starts to move
positions = [{
    'time': None,
    'position': state.dyna.current_state.position
}]
pickle.dump(state, open(os.path.join(run_folder, f"state_{0:05d}.pkl"), "wb"))
n_saved_states = 1

positions[0]['time'] = state.time

print("Start to record data")

start_time = state.time

while True:
    
    state = interface.get_simulation_state()
    
    if (state.time - start_time) / save_position_every > len(positions):
        positions.append({
            'time': state.time,
            'position': state.dyna.current_state.position
        })
        
    if (state.time - start_time) / save_state_every > n_saved_states:
        pickle.dump(state, open(os.path.join(run_folder, f"state_{n_saved_states:05d}.pkl"), "wb"))
        n_saved_states += 1

    time.sleep(.1)

    if keyboard.is_pressed("q"):
        print("Keybord Interrupt")
        break

    if state.player_info.race_finished :
        print("Race Finished")
        break
        
pickle.dump(positions, open(os.path.join(run_folder, "positions.pkl"), "wb"))

KeyboardInterrupt: 

## Rewind the game to a specific state

In [33]:
run_folder = "track_data/Training_dataset_flat_tech/run-2"
state_files = list(filter(lambda x: x.startswith("state"), os.listdir(run_folder)))
# random.shuffle(state_files)


for state_file in state_files :
    state = pickle.load(open(os.path.join(run_folder, state_file), "rb"))
    print(state_file)
    state.dyna.current_state.linear_speed = np.array([0, 0, 0])
    interface.rewind_to_state(state)
    time.sleep(1)

# for k in range(3):
#     state_file = state_files[8]
#     print(state_file)
#     state = pickle.load(open(os.path.join(run_folder, state_file), "rb"))
#     state.dyna.current_state.linear_speed = np.array([0, 0, 0])
#     interface.rewind_to_state(state)
#     time.sleep(1)

state_00000.pkl
state_00001.pkl
state_00002.pkl
state_00003.pkl
state_00004.pkl
state_00006.pkl
state_00008.pkl
state_00009.pkl
state_00011.pkl
state_00012.pkl
state_00014.pkl
state_00015.pkl


## Load positions

In [18]:
run_folder = "track_data/Straight_Line/run-2"
positions = pickle.load(open(os.path.join(run_folder, "positions.pkl"), "rb"))
print(positions)

[{'time': 10, 'position': [ArrayFieldProxy object at 0x25fe4c7b6b0]
Data: [820.7999877929688 10.210000038146973 592.0]}, {'time': 2600, 'position': [ArrayFieldProxy object at 0x25fe4d55af0]
Data: [820.7999877929688 10.210000038146973 592.0]}, {'time': 2720, 'position': [ArrayFieldProxy object at 0x25fe4d55f70]
Data: [820.7144165039062 10.05434799194336 592.0]}, {'time': 2820, 'position': [ArrayFieldProxy object at 0x25fe4d55ca0]
Data: [820.4713134765625 10.006918907165527 592.0006713867188]}, {'time': 2920, 'position': [ArrayFieldProxy object at 0x25fe4d55910]
Data: [820.0684814453125 10.008275985717773 592.000732421875]}, {'time': 3030, 'position': [ArrayFieldProxy object at 0x25fe4d55940]
Data: [819.441650390625 10.010725021362305 592.000732421875]}, {'time': 3130, 'position': [ArrayFieldProxy object at 0x25fe4d55a30]
Data: [818.7066650390625 10.011608123779297 592.000732421875]}, {'time': 3230, 'position': [ArrayFieldProxy object at 0x25fe4d55ac0]
Data: [817.8150024414062 10.0104780

In [24]:
list(state.dyna.current_state.position.to_numpy())

[1012.8001708984375, 10.013981819152832, 624.001220703125]

# Alternative Checkpoint Saving
You have to run the map, 
restart at each checkpoint without making inputs, 
and press "p" for picture, which saves the current state of the game

In [9]:
run_folder = create_folder_path(track_name=track_name)

n_saved_states = 0

print("Start to record data")

while True:
    
    state = interface.get_simulation_state()

    if keyboard.is_pressed("p"):

        pickle.dump(state, open(os.path.join(run_folder, f"state_{n_saved_states:05d}.pkl"), "wb"))
        print("Recorded state:", f"state_{n_saved_states:05d}.pkl")
        n_saved_states += 1
    
    time.sleep(.1)

    if keyboard.is_pressed("q"):
        print("Stop recording")
        break

Start to record data
Recorded state: state_00000.pkl
Recorded state: state_00001.pkl
Recorded state: state_00002.pkl
Recorded state: state_00003.pkl
Recorded state: state_00004.pkl
Recorded state: state_00005.pkl
Recorded state: state_00006.pkl
Recorded state: state_00007.pkl
Recorded state: state_00008.pkl
Recorded state: state_00009.pkl
Recorded state: state_00010.pkl
Recorded state: state_00011.pkl
Recorded state: state_00012.pkl
Recorded state: state_00013.pkl
Recorded state: state_00014.pkl
Recorded state: state_00015.pkl
Stop recording


# Testing

False